# Hands on Dash : 

## 1. Explore SVM with plotly viz

For this tuto, We will use 
[sklearn classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py)
 as our baseline to draw performance metrics of the SVM classifier when trained on Moons' data

In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

### create dataset

In [2]:
from sklearn import datasets
X, y = datasets.make_moons(n_samples=500, noise=0.3, random_state=0)

### plot initial X and y

In [3]:
colors = [[0, 'red'], [1, 'blue']]

In [4]:
data_points = go.Scatter(
    x=X[:, 0],
    y=X[:, 1],
    mode='markers',
    marker=dict(color=y, size=10, colorscale=colors)
)

In [5]:
iplot([data_points])

### train / test split

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [7]:
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

### model fit

In [8]:
from sklearn.svm import SVC

In [9]:
clf = SVC(kernel='rbf', gamma=2, C=1)
clf.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
score = clf.score(X_test, y_test)

In [11]:
"wahoo, you scored {} !".format(score)

'wahoo, you scored 0.885 !'

### plot metrics
##### ROC AUC

In [12]:
from sklearn import metrics

In [13]:
decision_test = clf.decision_function(X_test)
fpr, tpr, threshold = metrics.roc_curve(y_test, decision_test)

    # AUC Score
auc_score = metrics.roc_auc_score(y_true=y_test, y_score=decision_test)

auc = go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name='Test Diata',
)

In [14]:
iplot([auc])

##### Confusion Matrix

In [15]:
y_pred_test = clf.predict(X_test)

matrix = metrics.confusion_matrix(y_true=y_test, y_pred=y_pred_test)
tn, fp, fn, tp = matrix.ravel()

values = [tp, fn, fp, tn]
label_text = ["True Positive",
              "False Negative",
              "False Positive",
              "True Negative"]
labels = ["TP", "FN", "FP", "TN"]

colors = ['#66c2ff', '#0000cc', '#c65353', '#ff9999']

confusion = go.Pie(
    labels=label_text,
    values=values,
    hoverinfo='label+value+percent',
    textinfo='text+value',
    text=labels,
    sort=False,
    marker=dict(
        colors=colors
    )
)

In [16]:
iplot([confusion])

##### Contours

In [17]:
import numpy as np

In [18]:
colors = [[0, 'red'], [1, 'blue']]
cm_bright = [[0, '#ff9999'], [1, '#66c2ff']]

In [23]:
h = 0.5
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])

contours = go.Contour(
    x=np.arange(xx.min(), xx.max(), h),
    y=np.arange(yy.min(), yy.max(), h),
    z=Z.reshape(xx.shape),
    colorscale=cm_bright,
)
test_points = go.Scatter(
    x=X_test[:, 0],
    y=X_test[:, 1],
    mode='markers',
    marker=dict(color=y_test, size=10, colorscale=colors, symbol='triangle-up')
)

In [24]:
iplot([contours, test_points])